In [288]:
import configparser
from pathlib import Path
import collections
import pandas as pd
import pandas.io.sql as pandasql
from psycopg2.extras import execute_values
from psycopg2 import connect
import numpy
import matplotlib as mpl
from IPython.display import HTML, display
import rick
import numpy as np
import matplotlib.pyplot as plt
from setuptools import setup, find_packages
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [410]:
merge_set = pd.read_sql('''select rank, start_vid, end_vid, length, link_set, new_start_vid, new_end_vid, new_length, new_link_set, st_hausdorffdistance  from congestion.route_nextbest_ordered 
                          limit 10
                          ''',con)

In [411]:
def check_usage(link_set, link_used):
    result = []
    #print('link_set' +str(link_set))
    #print('link_used' +str(link_used))
    if link_used == []:
        return False
    for i in link_used:
        if collections.Counter(link_set) == collections.Counter(i):
            result.append(True)
        else:
            result.append(False)
    if True in result:
        return True
    else:
        return False

In [415]:
link_used = []
new_sets = []
rows = []
unique_set = merge_set[['start_vid', 'end_vid']].drop_duplicates()

# loop through unique sets of intersection
# !!this will be ordered by priority and length eventually!!
for index, key in unique_set.reset_index(drop=True).iterrows(): 
    
    start_vid = unique_set['start_vid'].iloc[index]
    end_vid = unique_set['end_vid'].iloc[index]
    #print(start_vid, end_vid)
    prepare_set = merge_set[(merge_set['start_vid'] == start_vid) & (merge_set['end_vid'] == end_vid)]
    link_set = prepare_set['link_set'].iloc[0]
    
    # Check if this link_set was used to match another set before
    # if not, selection best and merge
    if check_usage(link_set, link_used) == False:
        # if this intersection has an next best one: 
        if prepare_set['new_start_vid'] is not None:
            
            # if there are more than one rank and length is less than 100 or something         
            if prepare_set['length'].iloc[0] < 100:    
                # if there are two possibility for this intersection set
                # then compare which one is better
                if prepare_set['rank'].shape[0] == 2: 
                    rank1=prepare_set[(merge_set['rank']==1)]
                    rank2=prepare_set[(merge_set['rank']==2)]
                    print(prepare_set['rank'].shape[0])
                    # find hausdorffdistance difference and length difference
                    hausorffdiff = abs(rank1['st_hausdorffdistance'].iloc[0]/rank2['st_hausdorffdistance'].iloc[0])
                    lengthdiff = rank1['new_length'].iloc[0] - rank2['new_length'].iloc[0]

                    # make selection based on hausdorffdistance difference and length difference
                    # could try and play with the params a little bit
                    if hausorffdiff < 5 and lengthdiff > 20:
                        select_set = 2
                    else:
                        select_set = 1

                    selection = merge_set[(merge_set['rank']==select_set)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]

                    # find links that were used to merge, append it to another list
                    # also have to find out if it was using a set of link that was used before
                    old_links = selection['link_set'].iloc[0]
                    new_links = selection['new_link_set'].iloc[0]
                    additional_links = list(set(new_links).difference(old_links))

                    if check_usage(additional_links,link_used) == False:
                        new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])
                        link_used.append(additional_links)
                        link_used.append(old_links)
                        new_sets.append(new_set)
                    else:
                        new_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
                        link_used.append(old_links)
                        new_sets.append(new_set)

                # if there is only one possibility
                # see if that possibility is viable (this has not been added)

                elif prepare_set['rank'].shape[0] == 1:
                    # if st_hausdorffdistance < maybe like 50 or something????: then use it, else append
                    selection = merge_set[(merge_set['rank']==1)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]
                    old_links = selection['link_set'].iloc[0]
                    new_links = selection['new_link_set'].iloc[0]
                    additional_links = list(set(new_links).difference(old_links))

                    if check_usage(additional_links, link_used) == False:
                        new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])
                        link_used.append(old_links)
                        link_used.append(additional_links)
                        new_sets.append(new_set)
                    else:
                        # if it was used, maybe this can be added to that merged segment too? 
                        # if its too long it can just get partition up later
                        new_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
                        print('was used before')
                        link_used.append(old_links)
                        new_sets.append(new_set)

            # if the set length is longer than 100 then 
            # check to see if it was used to merge 
            # if yes dont append, if not append 
            else: 
                print('longer than 100m')
                old_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
                new_sets.append(old_set)
                link_used.append(link_set)
                
        # if it was not used and does not have next best, append it 
        else:
            print('does not have next best')
            old_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
            new_sets.append(old_set)
            link_used.append(link_set)
    # if link_set as used, continue and dont append
    else:
        print('used')

for i in range(len(new_sets)):
    item = new_sets[i]
    length = item[2].astype(float)
    row = (item[0].astype(float), item[1].astype(float), length, item[3])
    rows.append(row)
        
sql = '''insert into congestion.test_shortseg(start_vid, end_vid, length, link_set) VALUES %s'''    
with con:
    with con.cursor() as cur:
        execute_values(cur, sql, rows)    

        

109555486 30420391
2
30420391 109555486
2
30460238 30460237
2
30460237 30460238
2
30431761 

/etc/jupyterhub/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:26: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/etc/jupyterhub/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



30431762
2
30431762 30431761
2
30415731 30415730
2


IndexError: single positional indexer is out-of-bounds

In [409]:
len(new_sets)

3603